In [1]:
import cv2
import numpy as np

In [2]:
def process_frame(frame, bg_gray):
    # Converter para escala de cinzentos e suavizar
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Diferença para o background
    diff = cv2.absdiff(gray, bg_gray)

    # Threshold para regiões activas
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Limpeza morfológica
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,  kernel, iterations=1)

    # Encontrar contornos
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Maior contorno = corpo
        person_cnt = max(contours, key=cv2.contourArea)

        # Desenhar contorno do corpo (bordas dos limbs)
        cv2.drawContours(frame, [person_cnt], -1, (0, 255, 0), 2)

        if cv2.contourArea(person_cnt) > 5000:
            # Centroide
            M = cv2.moments(person_cnt)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])

                # Desenhar centroide (debug)
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

                # Pontos do contorno
                pts = person_cnt.reshape(-1, 2)  # (N, 2)

                # Encontrar ponto mais distante do centroide (candidato a mão)
                dx = pts[:, 0] - cx
                dy = pts[:, 1] - cy
                dists = np.hypot(dx, dy)

                max_idx = np.argmax(dists)
                hand_x, hand_y = pts[max_idx]
                max_dist = dists[max_idx]

                # Obter bounding box do corpo para usar como escala
                x, y, w, h = cv2.boundingRect(person_cnt)

                # Condições para considerar "braço esticado"
                if max_dist > 0.3 * w:  # 30% da largura do corpo
                    angle_deg = np.degrees(np.arctan2(dy[max_idx], dx[max_idx]))  # -180..180

                    # Aproximadamente horizontal
                    is_horizontal = (abs(angle_deg) < 30) or (abs(abs(angle_deg) - 180) < 30)

                    if is_horizontal:
                        # Vetor unitário na direção centroide -> mão
                        ux = dx[max_idx] / max_dist
                        uy = dy[max_idx] / max_dist

                        # Ponto "ideal" no meio do braço (no contínuo)
                        frac = 0.5  # 0.5 = meio do braço
                        target_x = cx + ux * max_dist * frac
                        target_y = cy + uy * max_dist * frac

                        # Procurar no contorno o ponto mais próximo desse target
                        diff_tx = pts[:, 0] - target_x
                        diff_ty = pts[:, 1] - target_y
                        dist_to_target = np.hypot(diff_tx, diff_ty)
                        mid_idx = np.argmin(dist_to_target)

                        # Este ponto está MESMO no contorno
                        arm_cx, arm_cy = pts[mid_idx]

                        # Desenhar ponto do centro do braço (no contorno)
                        cv2.circle(frame, (arm_cx, arm_cy), 7, (255, 0, 0), -1)
                        cv2.putText(frame, "Braco esticado",
                                    (arm_cx + 10, arm_cy - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                    (255, 0, 0), 1, cv2.LINE_AA)

                        # (Opcional) desenhar tambem a "linha do braco" para debug
                        # cv2.line(frame, (cx, cy), (hand_x, hand_y), (255, 0, 0), 2)

    return frame

In [3]:
def AugmentedRealityVideo(video0, ini, fin):
    cap = cv2.VideoCapture(video0)
    if not cap.isOpened():
        raise IOError(f"Não foi possível abrir o vídeo: {video0}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps          = cap.get(cv2.CAP_PROP_FPS)
    width        = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height       = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out_name = "video1.avi"
    fourcc   = cv2.VideoWriter_fourcc(*"XVID")
    out      = cv2.VideoWriter(out_name, fourcc, fps, (width, height), True)

    start_bg_frame = max(0, ini - 1)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_bg_frame)
    ret, bg_frame = cap.read()
    if not ret:
        cap.release()
        out.release()
        raise RuntimeError("Não foi possível ler frame de background.")

    bg_gray = cv2.cvtColor(bg_frame, cv2.COLOR_BGR2GRAY)
    bg_gray = cv2.GaussianBlur(bg_gray, (5, 5), 0)

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        if i < ini or i > fin:
            continue

        processed = process_frame(frame, bg_gray)
        out.write(processed)

    cap.release()
    out.release()
    return out_name



def AugmentedRealityWebcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Não foi possível abrir a webcam.")

    ret, bg_frame = cap.read()
    if not ret:
        cap.release()
        raise RuntimeError("Não foi possível ler frame da webcam para background.")

    bg_gray = cv2.cvtColor(bg_frame, cv2.COLOR_BGR2GRAY)
    bg_gray = cv2.GaussianBlur(bg_gray, (5, 5), 0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        processed = process_frame(frame, bg_gray)

        cv2.imshow("Augmented Reality - Webcam", processed)

        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [4]:
realTime = True
if(realTime):
    AugmentedRealityWebcam()
else:
    video0 = "./video0.avi"
    ini = 0
    fin = 100
    out_video = AugmentedRealityVideo(video0, ini, fin)